In [ ]:
!pip install hazm

In [ ]:
import os
import json
import string
from hazm import Normalizer, word_tokenize, stopwords_list
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import NGram

In [ ]:
# Rename all news files
folder_path = "/content/Hamshahri"
files = sorted(os.listdir(folder_path))

for idx, filename in enumerate(files):
    old_path = os.path.join(folder_path, filename)
    new_filename = f"N{idx+1}.txt"
    new_path = os.path.join(folder_path, new_filename)
    os.rename(old_path, new_path)

#Preprocessing

*   Removing Stopwords
*   Normalizing Data




In [ ]:
# Paths
input_dir = "/content/Hamshahri"
output_dir = "/content/processed_news"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
normalizer = Normalizer()
stopwords = set(stopwords_list())

In [ ]:
persian_punctuations = "،؛؟«»."
all_punctuations = set(string.punctuation + persian_punctuations)
special_stopwords = ['..','...',':.','+تا','!؟']
for file_name in os.listdir(input_dir):
  if not file_name.endswith(".txt"):
        continue
  file_path = os.path.join(input_dir, file_name)
  with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
    try:
      data = json.loads(content)
    except:
      continue
    raw_text = data.get("text", "")
    # Normalize text, tokenize, and remove stopwords and punctuation
    normalized = normalizer.normalize(raw_text)
    tokens = word_tokenize(normalized)
    cleaned_tokens = [word for word in tokens if word not in stopwords and word not in all_punctuations and word not in special_stopwords]
    cleaned_text = ' '.join(cleaned_tokens)

    output_file_path = os.path.join(output_dir, file_name)
    with open(output_file_path, 'w', encoding='utf-8') as out_file:
      out_file.write(cleaned_text)


# Spark Sql

In [ ]:
spark = SparkSession.builder.master('local[*]').appName('Spark_SQL_Hamshahri').getOrCreate()

In [ ]:
directory = '/content/processed_news'
df = spark.read.text(directory)
df.show(5)

+--------------------+
|               value|
+--------------------+
|صبح روز پنجشنبه ۳...|
|بدینسان فضای عینی...|
|فوتبال مساله مستق...|
|نگاهی تازه بنادر ...|
|اشاره متن حاضر اد...|
+--------------------+
only showing top 5 rows



In [331]:
split_data = df.select(split(split(input_file_name(), "/")[5],'\\.')[0].alias("file_name"),
                       split(col("value"), " ").alias('words'))

In [332]:
split_data.show(5)

+---------+--------------------+
|file_name|               words|
+---------+--------------------+
|     N197|[صبح, روز, پنجشنب...|
|     N201|[بدینسان, فضای, ع...|
|     N305|[فوتبال, مساله, م...|
|     N403|[نگاهی, تازه, بنا...|
|     N115|[اشاره, متن, حاضر...|
+---------+--------------------+
only showing top 5 rows



#Uni-Gram

In [333]:
df_unigram_exploded = split_data.withColumn("word", explode("words"))

In [334]:
df_unigram_exploded.show(5)

+---------+--------------------+-------+
|file_name|               words|   word|
+---------+--------------------+-------+
|     N197|[صبح, روز, پنجشنب...|    صبح|
|     N197|[صبح, روز, پنجشنب...|    روز|
|     N197|[صبح, روز, پنجشنب...|پنجشنبه|
|     N197|[صبح, روز, پنجشنب...|     ۳۰|
|     N197|[صبح, روز, پنجشنب...|  خرداد|
+---------+--------------------+-------+
only showing top 5 rows



In [335]:
df_unigram_counts = df_unigram_exploded.groupBy('word','file_name').agg(count('*').alias("count"))

In [336]:
df_unigram_counts.show(5, truncate = False)

+--------+---------+-----+
|word    |file_name|count|
+--------+---------+-----+
|اساس    |N197     |4    |
|منطقه   |N197     |7    |
|وباوری  |N201     |1    |
|ذوق     |N201     |1    |
|درواقعیت|N201     |2    |
+--------+---------+-----+
only showing top 5 rows



In [337]:
df_unigram_formatted = df_unigram_counts.withColumn("formatted", concat(col("file_name"), lit("["), col("count").cast("string"), lit("]")))

In [338]:
df_unigram_formatted.show(5, truncate= False)

+--------+---------+-----+---------+
|word    |file_name|count|formatted|
+--------+---------+-----+---------+
|اساس    |N197     |4    |N197[4]  |
|منطقه   |N197     |7    |N197[7]  |
|وباوری  |N201     |1    |N201[1]  |
|ذوق     |N201     |1    |N201[1]  |
|درواقعیت|N201     |2    |N201[2]  |
+--------+---------+-----+---------+
only showing top 5 rows



In [339]:
df_unigram = df_unigram_formatted.groupBy('word').agg(collect_list('formatted').alias('news_info'))

In [340]:
df_unigram.show(10, truncate= False)

+--------------+---------+
|word          |news_info|
+--------------+---------+
|A             |[N526[1]]|
|ABU           |[N305[1]]|
|ARIDIJISHOMERO|[N516[1]]|
|Alberto       |[N183[1]]|
|Andreini      |[N288[1]]|
|Antilpe       |[N159[1]]|
|Aranda        |[N383[1]]|
|Arctostaphylos|[N448[1]]|
|Art           |[N288[1]]|
|BLACK         |[N159[1]]|
+--------------+---------+
only showing top 10 rows



#Bi-Gram

In [341]:
bigram_ngram = NGram(n=2, inputCol="words", outputCol="BiGrams")
bigram_data = bigram_ngram.transform(split_data)

In [ ]:
bigram_data.show(5, truncate=  False)

In [343]:
df_bigram_exploded = bigram_data.withColumn("bigram", explode("BiGrams"))

In [344]:
df_bigram_exploded.show(10)

+---------+--------------------+--------------------+---------------+
|file_name|               words|             BiGrams|         bigram|
+---------+--------------------+--------------------+---------------+
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|        صبح روز|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|    روز پنجشنبه|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|     پنجشنبه ۳۰|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|       ۳۰ خرداد|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|      خرداد ماه|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|      ماه امسال|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|امسال خیابانهای|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...| خیابانهای فرعی|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|   فرعی مرکزشهر|
|     N197|[صبح, روز, پنجشنب...|[صبح روز, روز پنج...|  مرکزشهر تهران|
+---------+--------------------+--------------------+---------------+
only showing top 10 

In [345]:
df_bigram_counts = df_bigram_exploded.groupBy('bigram','file_name').agg(count('*').alias("count"))

In [346]:
df_bigram_counts.show(5, truncate = False)

+-----------------+---------+-----+
|bigram           |file_name|count|
+-----------------+---------+-----+
|تازه پسرم        |N197     |1    |
|روشنایی درآورد   |N197     |1    |
|مطابق معیارهای   |N197     |1    |
|تحصیل فرزندان    |N197     |2    |
|ناچاری می‌پردازند|N197     |1    |
+-----------------+---------+-----+
only showing top 5 rows



In [347]:
df_bigram_formatted = df_bigram_counts.withColumn("formatted", concat(col("file_name"), lit("["), col("count").cast("string"), lit("]")))

In [348]:
df_bigram_formatted.show(5, truncate= False)

+-----------------+---------+-----+---------+
|bigram           |file_name|count|formatted|
+-----------------+---------+-----+---------+
|تازه پسرم        |N197     |1    |N197[1]  |
|روشنایی درآورد   |N197     |1    |N197[1]  |
|مطابق معیارهای   |N197     |1    |N197[1]  |
|تحصیل فرزندان    |N197     |2    |N197[2]  |
|ناچاری می‌پردازند|N197     |1    |N197[1]  |
+-----------------+---------+-----+---------+
only showing top 5 rows



In [349]:
df_bigram = df_bigram_formatted.groupBy('bigram').agg(collect_list('formatted').alias('news_info'))

In [350]:
df_bigram.show(10, truncate= False)

+------------------------------+---------+
|bigram                        |news_info|
+------------------------------+---------+
|ABU اتحادیه                   |[N305[1]]|
|Alberto Villamiza             |[N183[1]]|
|Andreini ۱۶۰۴۱۵۶۲             |[N288[1]]|
|Antilpe cervicapraنام         |[N159[1]]|
|Aranda کارگردان               |[N383[1]]|
|Arctostaphylos alpina         |[N448[1]]|
|BORAGINACEAE گیاه             |[N359[1]]|
|BUCKخانواده گاوسانان          |[N159[1]]|
|Caesalpinia Pulcherrimaخانواده|[N160[1]]|
|Cayman Ilands                 |[N216[1]]|
+------------------------------+---------+
only showing top 10 rows



#Tri-Gram

In [351]:
trigram_ngram = NGram(n=3, inputCol="words", outputCol="TriGrams")
trigram_data = trigram_ngram.transform(split_data)

In [ ]:
trigram_data.show(5, truncate= False)

In [353]:
df_trigram_exploded = trigram_data.withColumn("trigram", explode("TriGrams"))

In [354]:
df_trigram_exploded.show(5)

+---------+--------------------+--------------------+----------------+
|file_name|               words|            TriGrams|         trigram|
+---------+--------------------+--------------------+----------------+
|     N197|[صبح, روز, پنجشنب...|[صبح روز پنجشنبه,...| صبح روز پنجشنبه|
|     N197|[صبح, روز, پنجشنب...|[صبح روز پنجشنبه,...|  روز پنجشنبه ۳۰|
|     N197|[صبح, روز, پنجشنب...|[صبح روز پنجشنبه,...|پنجشنبه ۳۰ خرداد|
|     N197|[صبح, روز, پنجشنب...|[صبح روز پنجشنبه,...|    ۳۰ خرداد ماه|
|     N197|[صبح, روز, پنجشنب...|[صبح روز پنجشنبه,...| خرداد ماه امسال|
+---------+--------------------+--------------------+----------------+
only showing top 5 rows



In [355]:
df_trigram_counts = df_trigram_exploded.groupBy('trigram','file_name').agg(count('*').alias("count"))

In [356]:
df_trigram_counts.show(5, truncate= False)

+-----------------------+---------+-----+
|trigram                |file_name|count|
+-----------------------+---------+-----+
|طریق تلویزیون دنبال    |N197     |1    |
|تازه پسرم امتحان       |N197     |1    |
|فرزندانش مدرسه‌ای مطابق|N197     |1    |
|سپری کارشناس پدردو     |N197     |1    |
|وجه اضافی بازپرداخت    |N197     |1    |
+-----------------------+---------+-----+
only showing top 5 rows



In [357]:
df_trigram_formatted = df_trigram_counts.withColumn("formatted", concat(col("file_name"), lit("["), col("count").cast("string"), lit("]")))

In [358]:
df_trigram_formatted.show(5, truncate= False)

+-----------------------+---------+-----+---------+
|trigram                |file_name|count|formatted|
+-----------------------+---------+-----+---------+
|طریق تلویزیون دنبال    |N197     |1    |N197[1]  |
|تازه پسرم امتحان       |N197     |1    |N197[1]  |
|فرزندانش مدرسه‌ای مطابق|N197     |1    |N197[1]  |
|سپری کارشناس پدردو     |N197     |1    |N197[1]  |
|وجه اضافی بازپرداخت    |N197     |1    |N197[1]  |
+-----------------------+---------+-----+---------+
only showing top 5 rows



In [359]:
df_trigram = df_trigram_formatted.groupBy('trigram').agg(collect_list('formatted').alias('news_info'))

In [360]:
df_trigram.show(10, truncate= False)

+-----------------------------+------------------------------------+
|trigram                      |news_info                           |
+-----------------------------+------------------------------------+
|Alberto Villamiza خانم       |[N183[1]]                           |
|Arctostaphylos alpina خانواده|[N448[1]]                           |
|BLACK BUCKخانواده گاوسانان   |[N159[1]]                           |
|Bennett راببنت انگلیسی       |[N206[1]]                           |
|Bombs ویروسهای کامپیوتری     |[N408[1]]                           |
|Bovidae راسته زوج            |[N272[1], N548[1], N159[1], N446[1]]|
|C E ۶۴۵۰                     |[N196[1]]                           |
|Capra Falconeirنام انگلیسی   |[N446[1]]                           |
|Capra ibex نام               |[N272[1]]                           |
|Commedia نمایش واقع          |[N288[1]]                           |
+-----------------------------+------------------------------------+
only showing top 10 rows



#Top 2Gram

In [361]:
bigram_top_30 = df_bigram_formatted.groupBy('bigram').agg(count('formatted').alias('count')).orderBy('count', ascending = False)

In [362]:
bigram_top_30.show(30, truncate= False)

+---------------+-----+
|bigram         |count|
+---------------+-----+
|جمهوری اسلامی  |67   |
|خبرگزاری فرانسه|50   |
|خبرنگار همشهری |50   |
|گزارش خبرگزاری |49   |
|سال گذشته      |36   |
|سرویس ورزشی    |34   |
|سرویس سیاسی    |32   |
|سرویس شهری     |31   |
|خبرگزاری جمهوری|30   |
|روابط عمومی    |27   |
|گزارش خبرنگار  |26   |
|هفتم تیر       |25   |
|اسلامی ایران   |24   |
|انقلاب اسلامی  |23   |
|رسانه‌های خارجی|21   |
|شهرداری منطقه  |21   |
|شهرداری تهران  |21   |
|علاقه مندان    |20   |
|واحد رسانه‌های |20   |
|گزارش روابط    |18   |
|دانش آموزان    |18   |
|ریاست جمهوری   |18   |
|شهر تهران      |18   |
|جام ملتهای     |18   |
|سرویس اقتصادی  |18   |
|روز گذشته      |17   |
|دکتر بهشتی     |16   |
|تیم ملی        |16   |
|علمی فرهنگی    |15   |
|آموزش پرورش    |15   |
+---------------+-----+
only showing top 30 rows



#Top 3Gram

In [363]:
trigram_top_30 = df_trigram_formatted.groupBy('trigram').agg(count('formatted').alias('count')).orderBy('count', ascending = False)

In [364]:
trigram_top_30.show(30, truncate= False)

+--------------------------+-----+
|trigram                   |count|
+--------------------------+-----+
|خبرگزاری جمهوری اسلامی    |30   |
|گزارش خبرگزاری فرانسه     |28   |
|جمهوری اسلامی ایران       |24   |
|واحد رسانه‌های خارجی      |20   |
|گزارش روابط عمومی         |18   |
|گزارش خبرگزاری جمهوری     |16   |
|گزارش خبرنگار همشهری      |14   |
|سرویس علمی فرهنگی         |14   |
|مجلس شورای اسلامی         |14   |
|هاشمی رفسنجانی رئیس‌جمهوری|11   |
|آیت‌الله دکتر بهشتی       |11   |
|جام ملتهای اروپا          |10   |
|حزب جمهوری اسلامی         |10   |
|شهدای هفتم تیر            |10   |
|واحد مرکزی خبر            |10   |
|گزارش واحد مرکزی          |10   |
|فاجعه هفتم تیر            |9    |
|رسانه‌های خارجی همشهری    |9    |
|فوتبال جام ملتهای         |9    |
|روابط عمومی سازمان        |8    |
|موزه آثار طبیعی           |8    |
|تیم ملی فوتبال            |8    |
|حیات وحش ایران            |8    |
|رئیس قوه قضائیه           |8    |
|شهیدان هفتم تیر           |8    |
|جام ملتهای آسیا    